In [ ]:
import csv
import re

def open_csv():
    # csv파일을 연다!
    f = open('IMDBDataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    doc_list = []

    next(csvreader)
    for f in csvreader:
        line = re.compile("[^\w]").sub(' ', f[0].lower())
        doc_list.append(line.split())

    return doc_list

doc_list = open_csv()

In [ ]:
!pip install glove_python

     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700247 sha256=bbe84c5bad242edda51e2fc424484cb101eb034118c03ee9adc69ce934f5bcc9
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [ ]:
from glove import Corpus, Glove

corpus = Corpus()
# 같이 등장한다는 것의 기준을 5로 지정
corpus.fit(doc_list, window=5)

# 경사하강법을 사용하기 때문에 학습률을 설정, 아웃풋 벡터의 차원은 100
glove = Glove(no_components=100, learning_rate=0.05)
# 쓰레드 개수는 4개, 훈련 횟수는 20번, verbose 설명
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
# 유사도 검색을 위해서 행렬의 index 정보를 입력
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [ ]:
glove_result = glove.most_similar('dog')
print(glove_result)

[('spot', 0.9534997404936615), ('road', 0.9463646793520095), ('basic', 0.9407350386932332), ('creature', 0.9382841166227128)]


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

token = Tokenizer()
text = "Tensorflow tokenizer is good"

# 리스트 형태로 넣어주어야 제대로 토큰화가 된다 (아니면 한글자씩 자름)
# fit 한 것을 토대로 추후에 sequence를 생성한다.
token.fit_on_texts([text])

# 단어 집합
print(token.word_index)
# 단어 빈도
print(token.word_counts)

# 정수 인코딩
text_2 = "Tensorflow tokenizer is not good good"
int_encoding = token.texts_to_sequences([text_2])
print(int_encoding)

# 행렬화
text_list = ['안녕하세요 감사해요 잘있어요 다시 만나요','오늘도 기분 좋은 하루','안녕하세요 오늘도 감사해요']
token.fit_on_texts(text_list)

# count는 dtm, binary는 존재 여부, freq는 빈도율, tfidf는 tfidf
print("tfidf")
matrix1 = token.texts_to_matrix(text_list, mode='tfidf')
print(matrix1)
print("dtm")
matrix2 = token.texts_to_matrix(text_list, mode='count')
print(matrix2)
print("존재여부")
matrix3 = token.texts_to_matrix(text_list, mode='binary')
print(matrix3)
print("빈도")
matrix4 = token.texts_to_matrix(text_list, mode='freq')
print(matrix4)

{'tensorflow': 1, 'tokenizer': 2, 'is': 3, 'good': 4}
OrderedDict([('tensorflow', 1), ('tokenizer', 1), ('is', 1), ('good', 1)])
[[1, 2, 3, 4, 4]]
tfidf
[[0.         0.84729786 0.84729786 0.         0.         0.
  0.         0.         1.09861229 1.09861229 1.09861229 0.
  0.         0.        ]
 [0.         0.         0.         0.84729786 0.         0.
  0.         0.         0.         0.         0.         1.09861229
  1.09861229 1.09861229]
 [0.         0.84729786 0.84729786 0.84729786 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
dtm
[[0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
존재여부
[[0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
빈도
[[0.         0.2        0.2        0.         0.         0.
  0.         0.         0.2        0.2        0.2        0.

### 모델 사용

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

token = Tokenizer()
text = "Keras tokenizer is good"
word_count = len(text)
token.fit_on_texts([text])

In [17]:
import numpy as np

f = open('glove.6B.50d.txt', encoding='utf-8')

word_vectors = {}

for line in f:
    word_vector = line.split()
    word_vectors[word_vector[0]] = word_vector[1:]

matrix = np.zeros((word_count, 300))
# ex) 4*50 행렬이 만들어짐


# 없는 단어에 대한 예외처리
for word, i in token.word_index.items():
    matrix[i] = np.array(word_vectors['word'])

# 실제로 사용
model = Sequential()
model.add(Embedding(word_count, 50, weights=[matrix]))
# glove로 임베딩된 벡터 값들을 가지고 실제로 모델에서 값들을 사용하는 임베딩 층을 만든다.


### 구글에서 만든 word2Vec 모델은 bin 형태로 만들어져 있으므로 사용가능

In [ ]:
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load_word2vec_format('파일이름.bin.gz', binary=True)

matrix = np.zeros((word_count, 300))
# ex) 4*50 행렬이 만들어짐


# 없는 단어에 대한 예외처리
for word, i in token.word_index.items():
    matrix[i] = np.array(word_vectors['word'])

# 실제로 사용
model = Sequential()
model.add(Embedding(word_count, 50, weights=[matrix]))
# glove로 임베딩된 벡터 값들을 가지고 실제로 모델에서 값들을 사용하는 임베딩 층을 만든다.
